# Software Architecture

As a former software architect, I viewed my role as ensuring these things:

 - the software supported the things we needed it to do
 - constraints and conflicts among the software requirements were evident, as were the decisions made in resolving them
 - the team (including new joiners) understood the design choices, motivations behind them, and situations in which we'd review them
 - the software could easily be extended to support things we were planning

It's this last one that's the most contentious. Software engineers are rightly concerned that if [You Aren't Gonna Need It](http://c2.com/xp/YouArentGonnaNeedIt.html) (YAGNI), it's a waste of time building it (not just time wasted now, but later, in maintaining the unused code). Practices like Test-Driven Development enforce YAGNI by requiring that the only code you write is code you have tests for. Presumably, you have tests because you derived them from the requirements, and conversely if it's not required then it's not tested.

But knowing about YAGNI can lead engineers to avoid _planning_ for the future, even for things that are near certain to come about. If you know that in six months you'll get the grant to work on integrating with some popular Java packages, it makes very little sense to start working in C# now based on the idea that "You Ain't Gonna Need" JVM interoperability. If a change of policy coming into force next year means that all participant data must be stored onsite, putting your database into a public cloud today means guaranteed work - up against a deadline - later.

## Architecture Decisions

The outputs associated with most of the responsibilities of a software architect are decisions that represent constraints to be imposed on the implementation of the software. "Participant records will be stored in MongoDB" is a decision that constrains the use of storage technologies; "the graphical interface follows the Model-View-Controller design pattern" is a decision that constrains the detailed design of the GUI software.

Constraints are an important part of design, because by restricting the space of possible designs you open up creativity within the space. Additionally, your design constraints act as guides for engineers working with the software. If you come to a project where _no_ constraint exists on the composition of GUI elements, they could be _anywhere_. Indeed, they may end up in multiple places within the same project, as each engineer working on an independent task chooses the approach that they think is best.

This means that the decisions made should not be arbitrary. Indeed, they should never be arbitrary, as you will need to get support for the decisions from everybody involved. In commercial software architecture, the architect must make sure that the software can be implemented by the developers, verified and validated by the testers, deployed by operations, used by the customers, sold by sales and marketing, integrated by partners, and paid for by management. The list of people involved and their motivations will differ in a research software project, but the point remains: an architect needs to understand and balance the views of the whole team and involve them in a process that fairly arrives at decisions that help the team collectively.

Sometimes, that means making decisions _not_ to write the software. Choosing to use commercial off-the-shelf or open source software instead of building a capability yourself can get you your results faster and cheaper - unless the results you're after are to publish an evaluation of a custom implementation of an algorithm, or build up strategic expertise in the problem domain. Buying an existing product, getting contractors in, or doing the work in your own team are all good ways to get a software capability.

The ultimate framework that guides the decisions you make as an architect is the requirements you have for the software. We looked at requirements engineering in the last session. The constraints imposed on the software by architecture choices should enable the satisfaction of the requirements. For example, if the problem at hand calls for high-integrity audit logging, then one solution could be for all communication between components to be made via a Enterprise Service Bus, with messages logged when they arrive at the Bus. However, if this solution would stop the software from satisfying other requirements (maybe performance or confidentiality), then an alternative needs proposing.

### Recording Architecture Decisions

Making decisions is not much value if they are not followed, and they definitely won't be followed if nobody knows about them. It's important to keep a development team educated as to the current state of the architecture and changes as they happen, and to make it easy for new team members to get up to speed and find their way around the code. This means that the architecture needs to be documented.

Yes, the usual caveat about keeping documentation and software in sync apply, but research teams tend to be more willing to engage with documentation than commercial software teams. If your software is delivered as a Jupyter Notebook or Literate Haskell, then you have a handy place to keep rich documentation and software in sync. If not, then keeping documentation in the source repository is a good solution. It means that the documents are always available to developers, and are not too far away when they are changing the code increasing the likelihood that they remember to keep them up to date, or that somebody notices in code review that updating is necessary.

This means actually _in_ the repository. GitHub and other code hosting tools have wikis associated with repositories. The wikis in GitHub are even git repos themselves. But they are not the _source_ repo, and they are not version controlled with the source code. There is more likelihood that the information in there will drift out of sync with the source. Additionally, there are more actions needed to get from the source folder to the wiki, so less likelihood that someone will even think to look there.

Nat Pryce, one of the co-authors of _Growing Object-Oriented Software, Guided by Tests_, created `[adrtools](https://github.com/npryce/adr-tools)` for working with "Lightweight Architecture Decision Records". These are text files in a standard form where you record who decided what, when, and why. You can also link records, indicating that a decision supercedes, clarifies, or updates a previous one. Such a format works well when storing architecture decisions in the repository to which they relate.

### C4 Model

Software Architect and consultant Simon Brown likens the architecture documentation for a software project to a series of maps: at different scales, the amount of information you show or hide is different, and the things that you call attention to change, too. He introduced the [C4 Model](https://c4model.com/) for visualising software architecture, which defines four levels of detail.

1. _Context_ diagrams tell people a lot about the outside of a system, and nothing about the inside. The whole system is drawn as a box, with links to the people and other systems that interface with it. The context diagram calls out constraints like what data goes where, who needs to use the system, and what integrations exist or are planned.
2. _Container_ diagrams begin to break the application up into distinct high-level components, which can be physical (different parts are deployed to different servers, or into a user's web browser, for example) or logical (a web application has a "storage" service and a "reporting" service even if they're deployed at the same place, for example). At this level, it's possible to see how the functionality of the system is distributed across different parts, and how they work in concert to achieve the system's goals.
3. _Component_ diagrams dive into a particular high-level component to show its detailed design. Whether it is built with Object-Oriented or Functional Programming principles, the interfaces it exposes to communicate externally, and the modules needed within the component, can be shown in the component diagram.
4. _Code_ diagrams go deep into the detail of how a single component is implemented in code.

In practice, I have found that context and container diagrams are very useful to orient team members, and the context diagram can even become part of the marketing or conference collateral for talking about the system. Whether a team needs component or code diagrams, and whether they are long-lived or prepared _ad hoc_ when needed, seems to be much more flexible. Some teams abhor detailed design diagrams on the basis that they repeat a lot of information in the code and are unlikely to be kept up to date.

The idea of keeping diagrams in sync with the code brings up notions of CASE (computer-assisted software engineering) tools and round-trip diagram generation. These tools are not used much in practice, for the same reason that an aerial photo of a terrain is not a good substitute for a map. The story you tell with a diagram (the 1,000 words it's standing in for) is supported by the information you choose to bring forward or to hide on the diagram. Automated tools aren't going to provide the level of discernment needed to select the relevant details, and hide the irrelevant. And a diagram that can be round-trip engineered to and from the code necessarily contains all of the same information as the code, rendering it useless as a diagram.

## Walking Skeletons

High-level architectural choices are hard to make in a vacuum, but are also easiest to make when the project is a green field and fewer changes will be needed to accommodate the choice. This seemingly contradictory position can be resolved by making prototypes of your system that implement the architecture without all of the detailed behaviour. Such models are called "walking skeletons". You can use the walking skeleton to investigate how the architectural options you're considering respond.

## Architecture Astronauts

In 2001, Joel Spolsky defined the [architecture astronaut](https://www.joelonsoftware.com/2001/04/21/dont-let-architecture-astronauts-scare-you/) as someone who doesn't write any code, because they're too concerned with the architecture, and thinking about abstractions so high-level that they've gone above the oxygen layer into space.

I've met many developers who are wary of software architects by default because they've met, or heard of, too many architecture astronauts. But architecture and high-level design are critical to software engineering and to the success of a software project. So is having a happy, motivated team who are engaged with their work and interested in seeing the project succeed.

Invite the team to participate in architectural discussions and decision making, do not inscribe the architecture on tablets that are then handed down to the coders to implement. Involve people in prototyping and trying out new technologies. Be genuine about seeking feedback and criticism from colleagues. Drive, but don't enforce, consensus.